<a href="https://colab.research.google.com/github/isabelsanchez2-cpu/IA2025/blob/main/02_preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# --- 1. Cargar el archivo train.csv ---
df_train = pd.read_csv('/content/train.csv')
print('DataFrame cargado. Primeras 5 filas:')
display(df_train.head())

DataFrame cargado. Primeras 5 filas:


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


### Resumen de la Carga de Datos

Se ha cargado el archivo `train.csv` en un DataFrame de pandas llamado `df_train`. La salida muestra las primeras 5 filas del DataFrame, permitiendo una vista inicial de la estructura, las columnas y los tipos de datos presentes antes de cualquier modificación.

### Manejo de valores nulos e identificación de columnas duplicadas

Se imputan los valores nulos en columnas de tipo `object` con la moda y se elimina la columna `F_TIENEINTERNET.1` si es idéntica a `F_TIENEINTERNET`.

In [2]:
# Identificar columnas de tipo 'object' con valores nulos
object_cols_with_missing = [col for col in df_train.columns if df_train[col].dtype == 'object' and df_train[col].isnull().any()]

# Imputar valores nulos en columnas de tipo 'object' con la moda
for col in object_cols_with_missing:
    mode_value = df_train[col].mode()[0]
    df_train[col] = df_train[col].fillna(mode_value)

# Verificar si F_TIENEINTERNET y F_TIENEINTERNET.1 son idénticas y eliminar una
if 'F_TIENEINTERNET' in df_train.columns and 'F_TIENEINTERNET.1' in df_train.columns:
    if df_train['F_TIENEINTERNET'].equals(df_train['F_TIENEINTERNET.1']):
        df_train.drop('F_TIENEINTERNET.1', axis=1, inplace=True)
        print("Columna 'F_TIENEINTERNET.1' eliminada ya que es duplicada de 'F_TIENEINTERNET'.")

print('Valores nulos en columnas object imputados y columnas duplicadas manejadas.')

Columna 'F_TIENEINTERNET.1' eliminada ya que es duplicada de 'F_TIENEINTERNET'.
Valores nulos en columnas object imputados y columnas duplicadas manejadas.


### Resumen del Manejo de Nulos y Duplicados

En este paso, se han imputado los valores nulos para todas las columnas de tipo 'object' utilizando la moda (el valor más frecuente) de cada columna. Además, se detectó que la columna `F_TIENEINTERNET.1` era idéntica a `F_TIENEINTERNET` y, por lo tanto, fue eliminada para evitar redundancia en el dataset. El mensaje de salida confirma estas acciones.

### One-Hot Encoding para variables categóricas (excepto la variable objetivo)

Se transforman las columnas `object` restantes (excluyendo `RENDIMIENTO_GLOBAL`) en formato numérico usando one-hot encoding.

In [3]:
# Identificar columnas de tipo 'object' que no sean la columna objetivo 'RENDIMIENTO_GLOBAL'
object_cols_to_encode = [col for col in df_train.columns if df_train[col].dtype == 'object' and col != 'RENDIMIENTO_GLOBAL']

# Aplicar one-hot encoding a las columnas identificadas
df_train = pd.get_dummies(df_train, columns=object_cols_to_encode, drop_first=True, dtype=int)

print('One-Hot Encoding aplicado a variables categóricas.')

One-Hot Encoding aplicado a variables categóricas.


### Resumen del One-Hot Encoding

El one-hot encoding se ha aplicado a todas las columnas categóricas de tipo 'object', excepto a la columna objetivo `RENDIMIENTO_GLOBAL`. Esto ha transformado estas columnas en múltiples columnas binarias, donde cada nueva columna representa una categoría específica y contiene `0` o `1`. Como resultado, el DataFrame ha aumentado significativamente en el número de columnas (ahora tiene 1029 columnas). La columna `RENDIMIENTO_GLOBAL` sigue siendo de tipo 'object', ya que la codificaremos por separado.

### Label Encoding para la variable objetivo `RENDIMIENTO_GLOBAL`

Se convierte la variable objetivo categórica en un formato numérico y se elimina la columna original.

In [4]:
# Inicializar LabelEncoder
label_encoder = LabelEncoder()

# Aplicar Label Encoding a la columna 'RENDIMIENTO_GLOBAL'
df_train['RENDIMIENTO_GLOBAL_encoded'] = label_encoder.fit_transform(df_train['RENDIMIENTO_GLOBAL'])

# Eliminar la columna original 'RENDIMIENTO_GLOBAL'
if 'RENDIMIENTO_GLOBAL_encoded' in df_train.columns and 'RENDIMIENTO_GLOBAL' in df_train.columns:
    df_train.drop('RENDIMIENTO_GLOBAL', axis=1, inplace=True)
    print("Columna 'RENDIMIENTO_GLOBAL' eliminada después de la codificación.")

print('Label Encoding aplicado a la variable objetivo y columna original eliminada.')

Columna 'RENDIMIENTO_GLOBAL' eliminada después de la codificación.
Label Encoding aplicado a la variable objetivo y columna original eliminada.


### Resumen del Label Encoding de la Variable Objetivo

La columna `RENDIMIENTO_GLOBAL` ha sido codificada numéricamente utilizando `LabelEncoder`, creando una nueva columna `RENDIMIENTO_GLOBAL_encoded`. Cada categoría original (por ejemplo, 'alto', 'bajo') ahora está representada por un número entero. La columna original `RENDIMIENTO_GLOBAL` se ha eliminado del DataFrame, dejando solo su versión numérica codificada para el modelado.

### DataFrame finalizado después del preprocesamiento

Se muestran las primeras filas y la información general del DataFrame para verificar el resultado final.

In [5]:
print('\nPrimeras filas del DataFrame finalizado:')
display(df_train.head())

print('\nInformación general del DataFrame finalizado:')
df_train.info()


Primeras filas del DataFrame finalizado:


,ID,PERIODO_ACADEMICO,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,E_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,E_PRGM_ACADEMICO_ACUICULTURA,E_PRGM_ACADEMICO_ADMINISTRACION,E_PRGM_ACADEMICO_ADMINISTRACION FINANCIERA,...,F_EDUCACIONMADRE_No Aplica,F_EDUCACIONMADRE_No sabe,F_EDUCACIONMADRE_Postgrado,F_EDUCACIONMADRE_Primaria completa,F_EDUCACIONMADRE_Primaria incompleta,F_EDUCACIONMADRE_Secundaria (Bachillerato) completa,F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,F_EDUCACIONMADRE_Técnica o tecnológica completa,F_EDUCACIONMADRE_Técnica o tecnológica incompleta,RENDIMIENTO_GLOBAL_encoded
0,904256,20212,0.322,0.208,0.310,0.267,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
1,645256,20212,0.311,0.215,0.292,0.264,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,308367,20203,0.297,0.214,0.305,0.264,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,470353,20195,0.485,0.172,0.252,0.190,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,989032,20212,0.316,0.232,0.285,0.294,0,0,0,0,...,0,0,0,1,0,0,0,0,0,3



Información general del DataFrame finalizado:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121264 entries, 0 to 121263
Columns: 968 entries, ID to RENDIMIENTO_GLOBAL_encoded
dtypes: float64(4), int64(964)
memory usage: 895.6 MB


### Resumen del DataFrame Finalizado

El DataFrame `df_train` ahora está completamente preprocesado. Todas las columnas son de tipo numérico (`float64` o `int64`), lo que lo hace apto para el entrenamiento de modelos de Machine Learning. Las primeras filas (`head()`) muestran la nueva estructura con las columnas One-Hot Encoding y la variable objetivo codificada. La información general (`info()`) confirma que no quedan columnas de tipo 'object' y que no hay valores nulos en el DataFrame.

### Normalización de columnas numéricas

Se escalan las columnas numéricas (`float64`) a un rango entre 0 y 1 utilizando `MinMaxScaler`. Esto es beneficioso para algoritmos que son sensibles a la escala de las características.

In [6]:
from sklearn.preprocessing import MinMaxScaler

# Identificar columnas numéricas a normalizar
numerical_cols = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

# Inicializar MinMaxScaler
scaler = MinMaxScaler()

# Aplicar MinMaxScaler a las columnas numéricas
df_train[numerical_cols] = scaler.fit_transform(df_train[numerical_cols])

print('Normalización de columnas numéricas completada.')
print('\nPrimeras filas del DataFrame con columnas numéricas normalizadas:')
display(df_train[numerical_cols].head())

Normalización de columnas numéricas completada.

Primeras filas del DataFrame con columnas numéricas normalizadas:


,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,0.490107,0.427105,0.968750,0.804217
1,0.473364,0.441478,0.912500,0.795181
2,0.452055,0.439425,0.953125,0.795181
3,0.738204,0.353183,0.787500,0.572289
4,0.480974,0.476386,0.890625,0.885542


### Resumen de la Normalización de Columnas Numéricas

Las columnas `INDICADOR_1`, `INDICADOR_2`, `INDICADOR_3` e `INDICADOR_4` han sido escaladas utilizando `MinMaxScaler`. Ahora sus valores se encuentran en un rango entre 0 y 1, lo cual es útil para asegurar que ninguna característica domine el entrenamiento de un modelo debido a su escala original. La salida muestra las primeras filas de estas columnas con sus nuevos valores normalizados.

### DataFrame finalizado después de todo el preprocesamiento

Se muestran las primeras filas y la información general del DataFrame para verificar el resultado final de todas las operaciones de preprocesamiento.

In [7]:
print('\nPrimeras filas del DataFrame finalizado después de normalización:')
display(df_train.head())

print('\nInformación general del DataFrame finalizado después de normalización:')
df_train.info()


Primeras filas del DataFrame finalizado después de normalización:


,ID,PERIODO_ACADEMICO,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,E_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,E_PRGM_ACADEMICO_ACUICULTURA,E_PRGM_ACADEMICO_ADMINISTRACION,E_PRGM_ACADEMICO_ADMINISTRACION FINANCIERA,...,F_EDUCACIONMADRE_No Aplica,F_EDUCACIONMADRE_No sabe,F_EDUCACIONMADRE_Postgrado,F_EDUCACIONMADRE_Primaria completa,F_EDUCACIONMADRE_Primaria incompleta,F_EDUCACIONMADRE_Secundaria (Bachillerato) completa,F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,F_EDUCACIONMADRE_Técnica o tecnológica completa,F_EDUCACIONMADRE_Técnica o tecnológica incompleta,RENDIMIENTO_GLOBAL_encoded
0,904256,20212,0.490107,0.427105,0.968750,0.804217,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
1,645256,20212,0.473364,0.441478,0.912500,0.795181,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,308367,20203,0.452055,0.439425,0.953125,0.795181,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,470353,20195,0.738204,0.353183,0.787500,0.572289,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,989032,20212,0.480974,0.476386,0.890625,0.885542,0,0,0,0,...,0,0,0,1,0,0,0,0,0,3



Información general del DataFrame finalizado después de normalización:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121264 entries, 0 to 121263
Columns: 968 entries, ID to RENDIMIENTO_GLOBAL_encoded
dtypes: float64(4), int64(964)
memory usage: 895.6 MB


### Resumen Final del DataFrame Preprocesado

El DataFrame `df_train` ahora está completamente preprocesado, incluyendo:
- **Carga de datos**
- **Manejo de valores nulos** en columnas `object` (imputación por moda)
- **Manejo de columnas duplicadas**
- **One-Hot Encoding** para variables categóricas (excepto la objetivo)
- **Label Encoding** para la variable objetivo `RENDIMIENTO_GLOBAL` y eliminación de la columna original
- **Normalización** de las columnas numéricas (`INDICADOR_1`, `INDICADOR_2`, `INDICADOR_3`, `INDICADOR_4`) a un rango de 0 a 1.

Todas las columnas son ahora de tipo numérico (`float64` o `int64`), sin valores nulos, y listas para el entrenamiento de modelos de Machine Learning. El DataFrame final es adecuado para la mayoría de los algoritmos de aprendizaje automático.